# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv('previsao_de_renda.csv')

df.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'], inplace=True)
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].mean())

X = df.drop(columns=['renda'])
y = np.log(df['renda'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

num_features = ['idade', 'tempo_emprego', 'qt_pessoas_residencia', 'qtd_filhos']
cat_features = ['sexo', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

ridge_results = {}
for alpha in alphas:
    ridge = make_pipeline(preprocessor, Ridge(alpha=alpha))
    ridge.fit(X_train, y_train)
    y_pred = ridge.predict(X_test)
    ridge_results[alpha] = r2_score(y_test, y_pred)

print("R2 Scores para Ridge:")
for alpha, r2 in ridge_results.items():
    print(f"Alpha {alpha}: R² = {r2:.4f}")

lasso_results = {}
for alpha in alphas:
    lasso = make_pipeline(preprocessor, Lasso(alpha=alpha))
    lasso.fit(X_train, y_train)
    y_pred = lasso.predict(X_test)
    lasso_results[alpha] = r2_score(y_test, y_pred)

print("R2 Scores para LASSO:")
for alpha, r2 in lasso_results.items():
    print(f"Alpha {alpha}: R² = {r2:.4f}")

stepwise_model = Ridge(alpha=0.01)
sfs = SequentialFeatureSelector(stepwise_model, n_features_to_select='auto', direction='forward')
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)
sfs.fit(X_train_transformed, y_train)

selected_features = sfs.get_support()
X_train_selected = X_train_transformed[:, selected_features]
X_test_selected = X_test_transformed[:, selected_features]

stepwise_model.fit(X_train_selected, y_train)
y_pred = stepwise_model.predict(X_test_selected)
stepwise_r2 = r2_score(y_test, y_pred)
print(f"R² para Stepwise: {stepwise_r2:.4f}")

decision_tree = make_pipeline(preprocessor, DecisionTreeRegressor(random_state=42))
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)
dt_r2 = r2_score(y_test, y_pred_tree)
print(f"R² para Árvore de Regressão: {dt_r2:.4f}")

R2 Scores para Ridge:
Alpha 0: R² = 0.3470
Alpha 0.001: R² = 0.3484
Alpha 0.005: R² = 0.3484
Alpha 0.01: R² = 0.3484
Alpha 0.05: R² = 0.3484
Alpha 0.1: R² = 0.3484


C:\Users\Lopes\anaconda3\Lib\site-packages\sklearn\base.py:1473: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\Lopes\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\Lopes\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.884e+03, tolerance: 8.797e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinat

R2 Scores para LASSO:
Alpha 0: R² = 0.3484
Alpha 0.001: R² = 0.3487
Alpha 0.005: R² = 0.3470
Alpha 0.01: R² = 0.3434
Alpha 0.05: R² = 0.3111
Alpha 0.1: R² = 0.2485
R² para Stepwise: 0.3479
R² para Árvore de Regressão: 0.3016
